In [6]:
# Necessary libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.optimizers import RMSprop
# from keras.utils.data_utils import get_file
import keras
import random
import numpy as np

In [7]:
text = open('SherlockHolmes.txt').read().lower()
print('Given script has ' + str(len(text)) + ' characters')

Given script has 581862 characters


In [8]:
text = text[1302:]
for ch in ['0','1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '$', '%', '&', '~', '`', '(', ')', '*',
          '-', '/', ';', '@', '?', ':', '©', '¢', 'ã', '\xa0', '\n', '\r', '.']:
       if ch in text:
             text=text.replace(ch,' ')
print(set(text))                

{"'", 'x', 'é', 'k', 'i', 't', 'f', 'u', 'è', 'b', 'l', 'g', 'e', 'q', 'r', 'z', 'c', 'v', 'à', 'j', 'a', 'm', 'h', 'o', 'd', 'y', 'n', 's', 'p', ' ', ',', 'â', 'w'}


In [9]:
def window_transform(text, window_size, step_size):    
    inputs = []
    outputs = []
    
    n_batches = int((len(text)-window_size) / step_size)    
    
    for i in range(n_batches-1):
        a = text[i * step_size:((i * step_size) + window_size)]
        inputs.append(a)
        b = text[(i * step_size) + window_size]
        outputs.append(b)

    return inputs,outputs

# Calling the window function
window_size = 50
step_size = 3
inputs, outputs = window_transform(text, window_size, step_size)

In [10]:
chars = sorted(list(set(text)))
print(chars)

[' ', "'", ',', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'â', 'è', 'é']


In [11]:
# Encoding 
chars_to_indices = dict((c, i) for i, c in enumerate(chars))

# Decoding
indices_to_chars = dict((i, c) for i, c in enumerate(chars))

In [12]:
def encode_io_pairs(text, window_size, step_size):    
    num_chars = len(chars)
    
    # cut up text into character input/output pairs
    inputs, outputs = window_transform(text, window_size, step_size)
    
    # create empty vessels for one-hot encoded input/output
    X = np.zeros((len(inputs), window_size, num_chars), dtype=np.bool)
    y = np.zeros((len(inputs), num_chars), dtype=np.bool)
    
    # loop over inputs/outputs and tranform and store in X/y
    for i, sentence in enumerate(inputs):
        for t, char in enumerate(sentence):
            X[i, t, chars_to_indices[char]] = 1
        y[i, chars_to_indices[outputs[i]]] = 1
        
    return X,y

X, y = encode_io_pairs(text, window_size, step_size)

In [13]:
# Designing the model
model = Sequential()
model.add(LSTM(120, input_shape=(window_size, len(chars))))
model.add(Dropout(0.22))
model.add(Dense(len(chars), activation='linear'))
model.add(Dense(y.shape[1], activation='softmax'))

# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Subsetting data for an example
Xsmall = X[:20000,:,:]
ysmall = y[:20000,:]

# Model training
model.fit(Xsmall, ysmall, batch_size=500, epochs=10)

2025-06-24 12:57:30.933804: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/wolf/anaconda3/envs/deepl-env/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


2025-06-24 12:57:31.177654: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33000000 exceeds 10% of free system memory.


40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 406ms/step - loss: 3.2201
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 362ms/step - loss: 2.8573
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 348ms/step - loss: 2.8082
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 17s 428ms/step - loss: 2.7607
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 519ms/step - loss: 2.6564
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 319ms/step - loss: 2.5477
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 329ms/step - loss: 2.4599
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 324ms/step - loss: 2.4138
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 406ms/step - loss: 2.3756
Epoch 10/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 318ms/step - loss: 2.3456


In [14]:
def predict_next_chars(model, input_chars, num_to_predict):     
    # create output
    predicted_chars = ''
    for i in range(num_to_predict):
        # convert this round's predicted characters to numerical input    
        x_test = np.zeros((1, window_size, len(chars)))
        for t, char in enumerate(input_chars):
            x_test[0, t, chars_to_indices[char]] = 1.

        # make this round's prediction
        test_predict = model.predict(x_test,verbose = 0)[0]

        # translate numerical prediction back to characters
        r = np.argmax(test_predict)                           # predict class of each test input
        d = indices_to_chars[r] 

        # update predicted_chars and input
        predicted_chars+=d
        input_chars+=d
        input_chars = input_chars[1:]
    return predicted_chars

In [15]:
start = 89
num_to_predict = 10
input_chars = text[start: start + window_size]
print('Complete sequence:', text[start:start + window_size + num_to_predict])
print('Input sequence:', input_chars)
print('Output sequence:', predict_next_chars(model, input_chars, num_to_predict = num_to_predict))

Complete sequence: otion akin to love for irene adler  all emotions, and that o
Input sequence: otion akin to love for irene adler  all emotions, 
Output sequence: an  he  he
